# EVERYTHING TRAINING

### CONFIG VARS

In [2]:
from utils import *
from ultralytics import YOLO

In [2]:
DATASET_PATH="../Datasets/Dataset"
PREPROCESSED_PATH="../Preprocessed_Datasets/Dataset"
AUGMENTED_PATH="../Augmented_Datasets/Dataset"
AUGMENTED_PATH_TRAIN_EX="../Augmented_Datasets/Dataset_train" # Data exclusively for training, not validating
YOLO_PATH="../YOLO_Datasets/Dataset"
YOLO_PATH_TRAIN_EX="../YOLO_Datasets/Dataset_train" # Data exclusively for training, not validating

In [4]:
if not os.path.exists(AUGMENTED_PATH):
    os.makedirs(AUGMENTED_PATH)
if not os.path.exists(AUGMENTED_PATH_TRAIN_EX):
    os.makedirs(AUGMENTED_PATH_TRAIN_EX)

### AUGMENTATION

For this model we will apply the following augmentation techniques:
- Hue transformations (-100º to +100º)
- Contrast inversion (To simulate dark and light modes)

In [10]:
create_slices(DATASET_PATH, AUGMENTED_PATH, 3, 3, 0.2, 0.2)

We apply resize now and not before because we want all images, including tiles, to keep the image size that the model will take

In [5]:
resize_dataset_images(DATASET_PATH, AUGMENTED_PATH, 640, 360)

In [6]:
hue_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15, 100)

In [7]:
contrast_inversion_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15)

In [8]:
# Add the augmented data to the train exclusively folder
for file in os.listdir(AUGMENTED_PATH):
    shutil.copy(os.path.join(AUGMENTED_PATH, file), AUGMENTED_PATH_TRAIN_EX)

### FORMAT CONVERSION

Up to now, we have treated with labelme format datasets, but we need to convert it to YOLOv8 format if we want to train
a model, which has the following format:

```
YOLOv8_Dataset/
├── data.yaml
├── train/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
├── valid/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
└── test/ (OPTIONAL)
    ├── images/
    │   ├── img1.jpg
    │   ├── img2.jpg
    │   └── ...
    └── labels/
        ├── img1.txt
        ├── img2.txt
        └── ...
```

The format of the data.yml file is:
```
path: <path_to_dataset_root_dit>
train: <path_to_train_images>
val: <path_to_validation_images>
test: <path_to_test_images> (OPTIONAL)

nc: <number_of_classes>
names: ['class1', 'class2', 'class3', ...]
```

The labels for Instance segmentation have the following format for each annotation:
```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```

In [9]:
labelme_to_yolo(AUGMENTED_PATH_TRAIN_EX, YOLO_PATH_TRAIN_EX, 0.7,[
                    "WebIcon",
                    "Icon",
                    "Switch",
                    "BtnSq",
                    "BtnPill",
                    "BtnCirc",
                    "CheckboxChecked",
                    "CheckboxUnchecked",
                    "RadiobtnSelected",
                    "RadiobtnUnselected",
                    "TextInput",
                    "Dropdown",
                    "Link",
                    "Text",
                    "TabActive",
                    "TabInactive",
                    "Sidebar",
                    "Navbar",
                    "Container",
                    "Image",
                    "BrowserURLInput",
                    "Header",
                    "Toolbar",
                    "BrowserToolbar",
                    "Scrollbar",
                    "Application",
                    "Taskbar",
                    "Dock",
                ], "seg")

### TRAIN

We will perform fine-tuning over the mobile-sam model using the hyperparameter tuning provided by Ultralytics to get the
best results we can. Since this is a non-standard dataset in terms of object features it is not clear what are the values
we should use.

We will also configure the training to not do any augmentation over the train set

In [1]:
from ultralytics import YOLO
# Initialize the YOLO model
model = YOLO("yolov8s-seg")

  0%|          | 0.00/22.8M [00:00<?, ?B/s]

In [11]:
import torch
# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {} device'.format(device))

Using cuda device


In [3]:
# Tune hyperparameters on dataset for 30 epochs
model.train(data=f"{YOLO_PATH_TRAIN_EX}/data.yaml", workers=1, epochs=30, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, fliplr=0.0)

New https://pypi.org/project/ultralytics/8.0.209 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208  Python-3.9.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=../YOLO_Datasets/Dataset_train/data.yaml, epochs=30, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train90, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

KeyboardInterrupt: 

In [3]:
model = YOLO("runs/segment/tune2/weights/best.pt")

In [5]:
# Validate the model
metrics = model.val(workers=1)  # no arguments needed, dataset and settings remembered

NotImplementedError: WARNING  'SAM' model does not support 'val' mode for 'segment' task yet.

In [11]:
metrics.seg.map    # map50-95
metrics.seg.map50  # map50
metrics.seg.map75  # map75
metrics.seg.maps   # a list contains map50-95 of each category

array([    0.19274,     0.32869,     0.40419,     0.39927,       0.561,     0.28032,     0.10029,     0.16814,     0.26469,     0.19651,     0.35886,     0.31838,     0.15703])

In [12]:
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

array([    0.44217,     0.57988,      0.5628,     0.66923,     0.72803,     0.39053,     0.33478,     0.53026,     0.41717,     0.49925,     0.55777,     0.43547,      0.2592])

In [10]:
inference = model("../YOLO_Datasets/Elementlevel_train/val/images/Captura de pantalla (44)_2.jpg", save=True)


image 1/1 d:\Code\Screen2SOM-Training\Training\..\YOLO_Datasets\Elementlevel_train\val\images\Captura de pantalla (44)_2.jpg: 384x640 33 WebIcons, 4 Icons, 3 BtnPills, 3 TextInputs, 2 Links, 29.0ms
Speed: 3.0ms preprocess, 29.0ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\segment\predict


In [1]:
# Load Mobile-SAM
from ultralytics import SAM

model = SAM("mobile_sam.pt")

In [4]:
# Tune hyperparameters on dataset for 30 epochs
model.tune(data="../YOLO_Datasets/ElementLevel_train/data.yaml", workers=1, epochs=30, iterations=20, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, translate=0.0, scale=0.0, fliplr=0.0, mosaic=0.0)

Tuner: Initialized Tuner instance with 'tune_dir=runs\detect\tune2'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/20 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.0, 'hsv_s': 0.0, 'hsv_v': 0.0, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.0, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.0, 'mosaic': 0.0, 'mixup': 0.0, 'copy_paste': 0.0}


Saved runs\detect\tune2\tune_scatter_plots.png
Saved runs\detect\tune2\tune_fitness.png

Tuner: 1/20 iterations complete  (1157.84s)
Tuner: Results saved to runs\detect\tune2
Tuner: Best fitness=0.53156 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.75375, 'metrics/recall(B)': 0.57982, 'metrics/mAP50(B)': 0.6638, 'metrics/mAP50-95(B)': 0.51687, 'val/box_loss': 0.87578, 'val/cls_loss': 0.80861, 'val/dfl_loss': 0.83703, 'fitness': 0.53156}
Tuner: Best fitness model is runs\detect\train2
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs\detect\tune2\best_hyperparameters.yaml'

lr0: 0.01
lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
box: 7.5
cls: 0.5
dfl: 1.5
hsv_h: 0.0
hsv_s: 0.0
hsv_v: 0.0
degrees: 0.0
translate: 0.0
scale: 0.0
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.0
mosaic: 0.0
mixup: 0.0
copy_paste: 0.0

Tuner: Starting iteration 2/20 with hyperparameters: {'lr0': 0.00995, 'lrf